In [2]:
import ipywidgets as wg
from IPython.display import SVG

# Compilers

A compiler is a program that takes your code (like C or Go) and turns it into a file containing only 0's and 1's.  These 0's and 1's can then be run on a computer.  A compiler consists of 5 stages:  Lexing, Parsing, Semantic Analysis, Optimization, and Code Generation. 

<details><summary>What are interpreters?  How are they different from compilers?</summary>

An interpreter takes your code and just runs it without outputting a binary file.  It still does the same job of turning your code into 0's and 1's, however.  So running a binary file output from a compiler will be faster than running a source file through an interpreter.

However, that doesn't mean interpreters aren't still useful.  Look at this link to find out why:

https://www.quora.com/Why-do-we-use-interpreters-instead-of-using-compilers-for-everything-Is-platform-independence-the-main-reason

</details>

# Lexing

Imagine you wrote this program:

	while (true):
		print(i)
	
To a compiler, your program currently looks like an array of characters: 
	
	[ w, h, i, l, e, (, t, r, u, e, ), :, p, r, i, n, t, (, i, ) ]
	
As they are, the compiler has no idea what these characters mean.  The first part of the compiler is called the lexer.  It takes this stream of characters and turn it into a stream of tokens, like so:

	[ while, (, true, ), :, print, (, i, ) ]
	
A token is a blob of characters that has meaning in your programming language.  None of 'w', 'h', 'i', 'l', or 'e' have meaning individually, but 'while' does.  

All tokens have a value.  Most tokens values are identical to their names, like 'while', '(', and 'true'.  Other tokens, like strings, numbers, and variables have values different from their names.  Here's another example program:

	x=4;
	
and here is the token output when a lexer reads this program:

	[ ID='x', =, INT=4 ]
	
In practice, the token name ID usually denotes variable names, function names, and class names.

<details><summary>How do we know that 'while' is a WHILE token?  How does the lexer know not to output 5 ID tokens?</summary>

Lexers use the maximum munch rule.  The maximum munch rule means that the lexer will create the biggest tokens it possibly can.  So first it sees a 'w', and thinks "this could be an ID... I'll keep going".  Then it sees an h, an i, an l, an e, and a (.  Then it thinks "Ok, I know that 'while(' isn't a possible token, so the biggest token I can make is 'while'.

The lexer knows to make 'while' a WHILE token, and not an ID='while' token through precedence rules, which are hard-coded.  So when it sees a 'while', it thinks "this could be an ID, or it could be a WHILE token.  Since WHILE takes precedence over ID, I'll make this a WHILE".
</details> 

Now, how do we actually code a lexer?  In other words, how do we create a program that will clump characters into tokens?  The answer is with regular expressions, or regex.  Regex describe patterns of characters using 3 operators:  Grouping, Boolean OR, and Quantification.

	Grouping:  r"ab" matches the string 'ab'
	Boolean OR:  r"a|b" matches the string 'a', or the string 'b'
	Quantification:  r"a*" matches the string '', the string 'a', the string 'aa', ... to infinity a's
	
You may have seen other regex operators in Java or Python, like + or ?.  These operators are just shortcuts of the 3 operators above.  `a+` is equivalent to `aa*` and `a?` is equivalent to `a|`.

We use regex to decide whether some string matches a pattern.  If we had a big file called X.txt, and we wanted to know if it contained only numbers, we could do it with regular expressions in python like this:

```python
	import re
	
	regex01 = r"[0-9]+" 
	#shorthand for (0|1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)*.
	#There's two (0|1|2|3|4|5|6|7|8|9)'s because we want at least 1 digit.
	
	X = open('X.txt', 'r').read() 
	#The file X.txt has been read into the string X
	
	if re.match(regex01, X):
		print("Match")
	else:
		print("No match")
```
		
So if X.txt contained something like `98560134760976`, then "Match" would be printed out, but if it contained something like `3704974320ABC947523`, then "No match" would be printed out.  But rather than printing things out, a real lexer would call a function that would emit a token; something like `emitToken(tokenName, tokenValue)`.  For this example, rather than calling `print("Match")` we would call `emitToken(INT, 98560134760976 )`, which would emit an INT token object whose value was 98560134760976.
		
<details><summary>Can regular expressions match anything?</summary>
Regex can match a lot of patterns, but there are certain patterns it can't match that we need to make a real programming language.  Consider C, or Java.  These languages require parenthesis and curly braces to be balanced, which means for every '(' that opens up a new scope, there needs to be a ')' to close the scope.

Now see if you can think of a regular expression that can match '()', '(())', '((()))', '(((())))', etc, all the way up to an arbitrary number of parenthesis, _without_ matching anything unbalanced, like '(()', '(()))', etc.  Your first thought might be something like `(*)*`, but that would end up matching '(()' and '(()))'.  Next you might try something like `()|(())|((()))|(((())))|....` Which is a good effort.  You could make a regex expression that could match balanced parenthesis in the hundreds, thousands, millions, or beyond, but you can never create a regex expression that can match an arbitrary number of parenthesis.  But the parser will take care of that, so our lexer doesn't have to worry about it.

What about python?  It doesn't have parenthesis at all.

While python doesn't have parenthesis, it still uses 'balanced' whitespace in order to define scope, which ends up being the same problem.  
</details> 

# Parsing

Look at this python program:

	y = f(4)                                      #line 1
	....                                          #line 2-199
	def f(x):                                     #line 200
		if x != 4:                                 #line 201
			print("that's not my favorite number") #line 202
			explode()                              #line 203
		else:                                     #line 204
			print("I cannot accept this gift")     #line 205
			return 4                               #line 206
	def explode():                                #line 207
		return 1 / 0                               #line 208
	
	
This program, like most, does not execute in order, e.g executing line 1, then line 2, then line 3, etc.  It starts at line 1, then jumps to line 200, then line 201, then line 204, 205, 206, then back to 1 before continuing.  

When our lexer turns the above program into a token stream, we get:

	[ ID=y, =, ID=f, (, INT=4, ), .... ]
	
The structure of this token stream is linear.  Line 1, line 2, line 3, etc.  It doesn't give us any information as to the order of program execution.

A parser turns a stream of tokens into a tree of tokens.  A tree's structure is non-linear; it can tell us more about the order in which a program will execute.

Even simple programs usually require some kind of tree structure.  Here is another example:

	2+3*2+3

and its token stream:

	[ INT=2, +, INT=3, *, INT=2, +, INT=3 ]
	
If our computer tried to execute this program token by token, these would be the steps it would take:

	2
	2+
	2+3
	5
	5*
	5*2
	10
	10+
	10+3
	13
	
Considering that 2+3*2+3=11, this is not the behavior that we want.  Computers are stupid.  They don't know that multiplication comes before addition.  If we instead take our token stream, and run it through a parser that understands our tokens, we would get an output like this:

![basic_tree](pics/basic_tree.png)

Our computer will traverse this tree inorder.  In actual code, the parse tree would look like this:

	( +
		2
		( +
			( *
				3
				2
			)
		)
	)
	
<details><summary>Wait, this looks like Lisp/Scheme...</summary>
That's because it is, basically.  Putting the tree all on 1 line, you would get `(+ 2 (+ (* 3 2) 3 ) )`.  You could copy and paste this exact line into a lisp interpreter / compiler and it would run successfully.  

In Lisp, you are basically skipping lexing and parsing almost entirely by directly creating the parse tree.  Supposedly this makes Lisp more powerful.  By powerful I don't mean computationally powerful, as in it runs faster.  I mean linguistically powerful, as in you can express computational ideas in a much more concise manner than, say, C++, Java, or Python.  Of course, this is a very difficult thing to quantify, or prove, but there is a very devoted community of Lisp users that swear by its almighty power.

Another note is that since Lisp mostly skips lexing and parsing, it can be compiled faster than pretty much any other high-level language.  However, I can't remember where I found the benchmarks that prove it.
</details>
		
Notice now that the multiplication 3 * 2 will happen before the addition 2 + 3.  Now that the computer has explicit parenthesis to tell it the exact order of operations, the final answer will correctly come out to 11.

Once we have a parse tree for the input program, that program definitely has some semblance of sense.  It may still have errors in it, but the program is speaking a language that we understand.  Let's get a sense of where we are with some metaphorical compilations of English:

`asdfasfasdf`  This is complete gibberish, and means nothing.  Fails at the lexing stage.

`is was of I pancakes`  While each individual word (token) makes sense, they don't form a grammatically correct sentence.  Fails at the parsing stage.

`The goats mustache is Cameron Diaz`  Ok, we have a grammatically correct sentence, but we're missing something.  'The' goat?  What goat are we talking about?  Who is Cameron Diaz?  I don't know who they are, but I know a human can't be a goats mustache.  Fails at the semantic analysis stage.

`I must proceed at high velocity` Now we're getting somewhere.  Each token makes sense, and together they form a parse tree that tells us that 'I' absolutely need to move very fast.  This is a valid program, and should be able to run.  If the optimization or code generation stages fail, that's an error in the compiler, not an error in the users program.

So once we have created a parse tree for our input program, it is either a 'goats mustache' or a 'high velocity' program.

However, we still don't know how to create a parser.  To do that, we need to learn more about formal language theory.

`Formal Languages`  A formal language is the set of all possible sentences that follow some set of rules.  Validity is not disputed in formal languages.  Let's say you invent a language, and someone asks you 'Is this paragraph written in your language?'  If you can use the rules of your language to prove with 100% certainty that the paragraph is valid (it is written in your language) or invalid (it is not written in your language), then your language is a formal language.

Java and C are formal languages.  If you compile some text and the compiler throws an error, the text is not a valid program.  If the program compiles and runs, it is a valid program.

English is not a formal language.  English has rules, but there are definitely words and phrases that are arguably valid or invalid.

<details><summary>What if the program compiles, but has a runtime error?  Is the program valid?</summary>
Yes.  A runtime error, like a stack overflow, is a problem with the hardware of the machine.  If we had more memory, that error would not have occured.  So it's a problem with your language.
</details> 

`Formal Grammar` formal grammars are the rules of formal languages.  The formal grammar rules in our parser are what turn the token stream into a token tree.  Parsers use a specific kind of formal grammar called a context-free grammar.

`Context-Free Grammar` CFGs are a subset of formal grammars.  Formal grammars use backus nar form to describe their rules.  

Example of a CFG in backus nar form:

	S -> x | xT
	T -> y | z
	
Formal grammars do the same thing that regular expressions do: they look through strings and return either true or false depending on whether the pattern matches or not.  They also construct a tree from the linear input they are given.

The above grammar will match the strings 'x', 'xy', and 'xz', and won't match anything else.
	
All formal grammars have:

A set of `terminal symbols`.  A terminal symbol is the smallest unit of meaning in a language.  Think of terminal symbols as single words.  The terminal symbols in our example are x, y, and z.  The terminal symbols in our compiler will be the tokens from our lexer. 

A set of `nonterminal symbols`.  A non-terminal symbol is a symbol that can be broken down into a combination of nonterminal and terminal symbols.  In our example, S and T are non-terminal symbols.  S can be broken down into either x, or xT.  T can be broken down into either y or z. 

A set of `production rules`.  Production rules tell you how to break down nonterminal symbols.  S->x is a production, S->xT is a production, T->y is a production, and T->z is a production.  S-> x | xT is not a production, it is 2 productions.

A `start symbol`.  A start symbol is just the nonterminal where you start parsing.  It's either denoted with S, Start, or it should be clear enough without explicit statement.

Here's another example:

	S -> (S) | epsilon
	
epsilon is a stand in for the empty string.  This grammar can match nested balanced parenthesis, which regex can't do.


<details><summary>Couldn't we do the S and T example with regex?</summary>
Yes.  A regular expression is actually a kind of formal grammar, and more, its a kind of context-free grammar.

All regex expressions have an equivalent backus-nar form.

Example regex:

	([A-Za-z])([A-Za-z0-9*])
	
Example equivalent backus-nar form:

	S -> Alpha T
	
	T -> Alpha T | Num T | epsilon
	
	Alpha -> A|a|B|b|C|c|D|d|E|e|F|f|G|g|H|h|I|i|J|j|K|k|L|l|M|m|N|n|O|o|P|p|Q|q|R|r|S|s|T|t|U|u|V|v|W|w|X|x|Y|y|Z|z
	
	Num -> 0|1|2|3|4|5|6|7|8|9
	
Our lexer is just a formal grammar.  It's terminals are individual characters, it's non-terminals are the tokens it outputs. Its start symbol is all of the tokens OR'd together, but we don't care about it.

Yeah, we could combine regex with backus nar form (thereby combining the lexer and the parser), but its better to keep them separate for a multitude of reasons.  Most of those reasons can be boiled down to 'it makes the implementation look ugly'.  There's also the fact that separating it into a lexer and parser lets us pipeline the two parts easily.  Also, it makes the whole 'parsing english' metaphor clearer, and therefore easier to understand conceptually.

So really the difference between regex and CFGs is that regex is just a special kind of CFG that is really easy to express, since we can use regular expressions instead of backus nar form.
</details>

#Misc

http://trevorjim.com/python-is-not-context-free/

<details><summary>TODO</summary>
put your goats mustache example up at the very top.  Then at each stage, use that example to give them a sense of what this stage of the compiler does.  Also add these in:
optimization:  this stage is optional.  Makes code easier for computer to execute.  'I must proceed at high velocity' becomes 'Gotta go fast'.
code generation:  'Gotta go fast' becomes '010100100101001010010101001' (but make it accurate).  

Is there a specific name for the kinds of things a regular grammar can't accomplish?  Like it can't accomplish infinitely recursive structures, or it can't do infinite nesting, or it can't do uncountably infinite sets, or it can't do non-modulo sets, or something.  Just give a name to it.  As of now, it's hard to figure out whether you can write something as a regular expression or not.  You have to use the pumping lemma, I think.  Maybe explain that simply.  Would actually be useful.  'Can I write this as a regular expression'?  If yes, fantastic, an easy win.  If not, oh well, at least you didn't waste a bunch of time trying to think of a way to do it.

https://cs.stackexchange.com/questions/51189/ambiguity-vs-context-sensitivity
https://en.wikipedia.org/wiki/Ambiguous_grammar
https://stackoverflow.com/questions/14589346/is-c-context-free-or-context-sensitive 

Also want to ask:  'why is top down parser easier to debug?'  reference the above stack exchange thing.  Both top down and bottom up are written using backus nar form.

</details>


# Semantic Analysis

If a program passes semantic analysis, it is a valid program.  Some of the common features of semantic analyzers include:
```
Checking that all IDs are declared before use
Checking that all types are correct
Checking class inheritances are correct
Checking that classes and methods are only defined once
```
This doesn't list everything, because semantic analysis varies from language to language.

Are we checking all this stuff all at once?
No, its recommended that you go over the whole program checking IDs, then pass over it again checking all types, then pass over it again checking all inheritances, etc.  Some of these passes are easier to combine, but in general your program will be much less error prone if you do multiple passes.

What is a scope?
A scope is how long a variable lives.  All variables have a scope that they exist in, and do not exist outside of.

What defines the beginning and end of a scope?
Different things in different languages.  Things like curly braces, parenthesis, if statements, for and while loops, etc.

What is global scope?  What is the scope of a class or function?  What is the scope of fields in a class?

Can I define functions inside functions?  Can I define classes inside functions?

What is overriding?
Overriding means you redefine something.  If scope 2 is in scope 1, and scope 1 has x, scope 2 can see x.  But you can define a new x in scope 2, and now when you refer to x in scope 2, the x in scope 1 is ignored.  You couldn't do this in scope 1, because then when you refer to x, how would you know which x you're refering to?  Actually, some languages would let you do that, like python.  You can totally have int x, then string x in the same scope.  Everything between the first and second definition will refer to int x, and everything after that will refer to string x.

What is a symbol table?
Say you're in scope X.  The symbol table will hold all the variables that are currently defined in scope X.  Once you leave scope X, the symbol table will change.  Professor Aiken says a 'simple' symbol table can be a stack.  Is this the stack as in the stack from 61C?

What is the difference between static and dynamic scoping?
Static scoping is normal, dynamic scoping is weird.  Dynamic scoping has its place sometimes.

What exactly is a type?
A set of values and the operations you can do on those values.
An int is 32 bits, and the set of values for the int type is −2,147,483,648 to 2,147,483,647.
You can perform addition, subtraction, multiplication, division, etc, on integers.
An unsigned int is 32 bits, so the set of values for the int type is −2,147,483,648 to 2,147,483,647.
You can perform addition, subtraction, multiplication, division, etc, on integers.

An unsigned int is another type that is 32 bits, and the set of values for this type is 0 to 4,294,967,295.
It has the same operations as ints.

A string is a type that has the values of every possible string, so it has infinite values (or whenever you run out of memory).
Its operations are concatenation, splitting, etc.

Classes are just fancy types.  When you make a class, the set of values of the class are all possible combinations of the values of each of its fields.
The set of operations that you can perform on your class are the methods you define for that class.

What about functions?  Are those types?
Not really sure.  I want to say that a function isn't a type, it's an operator.  But you can assign a function to a variable like x.  Variable x has to have a type.  Look at this example:
def f(int a, int b, int c): return a+b+c
x = f
I want to say that x is a variable whose type is (int, int, int) -> int, and whose value is f. 

Are types fundamental?  Do you absolutely need types?
I don't really have an answer to this.  In assembly, nothing has types.  But if you added 'a'+4, that doesn't make any sense.  If there was no type checking, you could do it, which doesn't make any sense.  Typing is like defining the vector spaces that your functions and stuff work on.

What is type inference?  How does type inference work?  Why do people call it type checking?
It's sometimes used interchangibly with type checking because if you do type inference then you're also doing type checking.
It's where you guess the types of each thing.
We do type inference for each expression.  So at least once per line, maybe more.
First, we get the types of all the free variables in the expression.  A free variable is a variable that wasn't defined in the expression.  I think free is a bad name.  Unless the variable is being assigned to, it's a free variable.  In `x=4`, x is a bound variable.  In `x=y+4` y is a free variable, and x is a bound variable.  Just say we need to bind all the variables.
Perhaps you should explain the algorithm.  You start at the top of the AST, and go down.  When you get to a leaf, you add that leaf and its type to your 'type environment'.  You go through all the children of a node, adding their types to the type environment until you know all the types of its children.  Then you can hopefully deduce the type of the parent.  Why is this a stack?  It seems like it would be better as a linked list.  Or maybe its a stack of scopes, rather than a stack of current variables.  That would make sense.

What's this turnstile thing?  How is this different from an if-then thing?  The arrow?  What is O?
the turnstile means 'proves'.  I think it's the same thing as an implication (the arrow).  O is the type environment.  If we have a statement `y=x+4`, all we know from this statement is that x is a variable name.  To get y's type, we need more information.  We get it from the type environment O, which is the table that contains all thetypes for the current scope.

What are the big fraction things?  Are these big fraction things declarative, or are they actually proving stuff?
I'm pretty sure it's an actual proof.  The numerator has the hypothesis in it, and the denominator has the conclusion.

Why do we have both a turnstile and a big fraction?
I think the turnstile is for stuff that's already proved, and the fraction is used to do a proof.